In [ ]:
!pip install transformers
!pip install datasets
!pip install huggingface_hub

     |████████████████████████████████| 3.8 MB 8.9 MB/s 
     |████████████████████████████████| 895 kB 32.2 MB/s 
     |████████████████████████████████| 67 kB 1.3 MB/s 
     |████████████████████████████████| 596 kB 40.2 MB/s 
     |████████████████████████████████| 6.5 MB 41.8 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 312 kB 9.5 MB/s 
     |████████████████████████████████| 212 kB 52.3 MB/s 
     |████████████████████████████████| 134 kB 57.4 MB/s 
     |████████████████████████████████| 1.1 MB 53.4 MB/s 
     |████████████████████████████████| 127 kB 49.7 MB/s 
     |████████████████████████████████| 271 kB 55.0 MB/s 
     |████████████████████████████████| 94 kB 2.1 MB/s 
     |████████████████████████████████| 144 kB 49.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# Initialize the tokenizer
tokenizer = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")
# Initialize the model
model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base", num_labels = 3)

Downloading:   0%|          | 0.00/565 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/478M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight', 'lm_head.layer_norm.weight', 'roberta.pooler.dense.weight', 'lm_head.bias', 'lm_head.dense.bias', 'lm_head.decoder.bias', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base and

In [ ]:
from datasets import load_dataset, DatasetDict
tweets = load_dataset('chiarab/final-train', use_auth_token=True)

Using custom data configuration chiarab--final-train-656636e58980eed2


  0%|          | 0/1 [00:00<?, ?it/s]

Downloading:   0%|          | 0.00/447k [00:00<?, ?B/s]

  0%|          | 0/1 [00:00<?, ?it/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/csv/chiarab--final-train-656636e58980eed2/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
# Take random examples for train and validation
finetune_train = tweets['train'].shuffle(seed=1111).select(range(30))
#finetune_val = tweets['train'].shuffle(seed=1111).select(range(2219, 2695))
#finetune_test = tweets['train'].shuffle(seed=1111).select(range(2695, 3170))

In [ ]:
#tweets['train'].to_csv('file1.csv' , index = True)
print(finetune_train)
#print(tweets)


Dataset({
    features: ['OriginalTweet ', 'Sentiment'],
    num_rows: 30
})


In [ ]:
from torch.utils.data import DataLoader

train_finetune_dataloader = DataLoader(finetune_train, batch_size=16)
#eval_finetune_dataloader = DataLoader(finetune_val, batch_size=16)

print(train_finetune_dataloader)

In [ ]:
def get_target(val):
  print(val)
  vec = np.zeros((1,3))
  print(vec)
  vec[0][val] = 1
  return torch.from_numpy(vec)

In [ ]:
import numpy as np
import torch
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm.notebook import tqdm

num_epochs = 3
num_training_steps = 3 * len(train_finetune_dataloader)
optimizer = AdamW(model.parameters(), lr=5e-5, weight_decay=0.01)
lr_scheduler = get_linear_schedule_with_warmup(optimizer, num_warmup_steps=0, num_training_steps=num_training_steps)

best_val_loss = float("inf")
progress_bar = tqdm(range(num_training_steps))
for epoch in range(num_epochs):
    # training
    model.train()
    for batch_i, batch in enumerate(train_finetune_dataloader):
      print(batch['Sentiment'])
      input = batch['OriginalTweet '][1]
      labels = get_target(batch['Sentiment'])
      inputs = tokenizer(input, return_tensors="pt")
      outputs = model(**inputs, labels=labels)
      outputs.loss.backward()
      optimizer.step()
      lr_scheduler.step()
      progress_bar.update(1)

    # validation
    loss = 0
    model.eval()
    for batch_i, batch in enumerate(eval_finetune_dataloader):
        with torch.no_grad():
          input = batch['OriginalTweet '][0]
          labels = get_target(batch['Sentiment'])
          inputs = tokenizer(input, return_tensors="pt")
          outputs = model(**inputs, labels=labels)
        loss += outputs.loss
    
    avg_val_loss = loss / len(finetune_val)
    print(f"Validation loss: {avg_val_loss}")
    if avg_val_loss < best_val_loss:
        print("Saving checkpoint!")
        best_val_loss = avg_val_loss
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'val_loss': best_val_loss,
            },
            f"epoch_{epoch}.pt"
        ) 


/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/6 [00:00<?, ?it/s]

tensor([0., 2., 1., 1., 0., 1., 1., 2., 1., 2., 0., 2., 2., 1., 1., 1.],
       dtype=torch.float64)
tensor([0., 2., 1., 1., 0., 1., 1., 2., 1., 2., 0., 2., 2., 1., 1., 1.],
       dtype=torch.float64)
[[0. 0. 0.]]


IndexError: ignored

In [ ]:
#our trained model accuracy

correct = 0
wrong = 0

model.eval()
for tweet in finetune_test:
  inputs = tweet['id']
  tokenized_inputs = tokenizer(inputs, return_tensors="pt")
  outputs = model(**tokenized_inputs)
  labels = [0, 1, 2]
  prediction = torch.argmax(outputs.logits)
  if prediction == tweet['sentiment_category']:
    correct += 1
  else:
    wrong += 1

print("Finetuned Accuracy:")
print(correct / (correct + wrong))

In [ ]:
correct = 0
wrong = 0

# Initialize the regular tokenizer
tokenizer2 = AutoTokenizer.from_pretrained("cardiffnlp/twitter-roberta-base")
# Initialize the regular model
model2 = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base", num_labels = 3)

model2.eval()
for tweet in finetune_test:
  inputs = tweet['id']
  tokenized_inputs = tokenizer2(inputs, return_tensors="pt")
  outputs = model2(**tokenized_inputs)
  labels = [0, 1, 2]
  prediction = torch.argmax(outputs.logits)
  if prediction == tweet['sentiment_category']:
    correct += 1
  else:
    wrong += 1

print("Regular Accuracy:")
print(correct / (correct + wrong))
print(correct + wrong)